# Algorithmen

In [ ]:
%run ./Muehle_Logic.ipynb

In [ ]:
%run ./Muehle_Utilities.ipynb

In [ ]:
%run ./Muehle_Heuristik.ipynb

In [ ]:
def memoize(f):
    global Cache
    
    def f_memoized(*args):
        key = (args[0], args[1])
        if key in Cache:
            return Cache[key]
        result = f(*args)
        Cache[key] = result
        return result
    
    return f_memoized

## Minimax-Algorithmus

Der Minimax-Algorithmus wird im Rahmen dieser Studienarbeit für die Ermittlung der optimalen Strategie für das Nulsummenspiel Mühle verwendet. 
Prinzip. Dabei ist der Minimax-Algorithmus ist ein relativ einfacher Algorithmus, der durch die Verwendung von Alpha-Beta-Pruning deutlich verbessert wird.

Der Minimax-Algorithmus beruht auf einer Bewertungsfunktion und systematischer Suche. Gundlegend werden bei Minimax alle auf den Aktuellen Spielstatus folgenden Zustände berechnet und bewertet. Dies ist vergleichbar mit einer Baumstruktur, bei der bis zu den Blättern alle Zustände ausgewertet werden. Da dies aus Gründen der Rechenzeit und des Speichers nicht möglich ist, werden die Folgezustände nur bis zu einer gewissen Tiefe berechnet und ausgewertet. Um nur bis zu einer geringen Baumtiefe suchen zu können wird allerdings eine geeignete Heuristik benötigt. Mit der Nutzung dieser Heuristik verlieren wir allerdings die Sicherheit den optimalen Zug zu wählen.

In [ ]:
import random
random.seed(1)

Die Funktion `value_minimax(State, player)` erhält drei Argumente. Einen Spielzustand, einen Spieler und die Suchtiefe. Die Funktion gibt dabei den Wert zurück, den dieser Status für den übergeben Spieler hat.
Dieser Wert Wert wird, für den Fall, dass das Spiel mit dem übergebenen Zustand beendet ist von def Funktion `finished()` berechnet. Wenn die maximale Suchtiefe erreicht wurde wird der Wert allerdings von der Funktion `heuristic()` berechnet.
Ist die maximale Suchtiefe noch nicht erreicht wird rekursiv nach dem besten Folgezustand gesucht.
Um Werte für Zustände nicht mehrfach berechnen zu müssen werden diese duch Memoisation `@memoize` zwischengespeichert.

In [ ]:
@memoize
def value_minimax(state, player, depth):
    if finished(to_list(state)):
        return utility(to_list(state), player)
    if depth == 0:
        return heuristic(state, player)
    o = opponent(player)
    depth -= 1
    return max([ -value_minimax(to_tuple(ns), o, depth) for ns in next_states(to_list(state), player) ])

Die Funktion `best_move_minimax(State, player)` erhält drei Argumente. Einen SPielzustand, einen Spieler und die Suchtiefe. Die Rügkabewerte sind der von der Funktion ermittelte beste Folgezustand und dessen Bewertung. Gibt es mehrere beste Folgezustände wird der Folgezustand zufällig ausgewählt.

In [ ]:
def best_move_minimax(state, player, depth):
    ns          = next_states(state, player)
    best_value  = value_minimax(to_tuple(state), player, depth)
    best_moves  = [s for s in ns if -value_minimax(to_tuple(s), opponent(player), depth - 1 ) == best_value]
    best_state  = random.choice(best_moves)
    return best_value, best_state

Die Funktion minimax(State, Player) wurde erstellt, um die Funktion best_move_minimax(State, player) nach Außen eindeutiger von alpha_beta_pruning(State, Player) abzugrenzen. Die übergebenen Argumente und Rückgabewerte entsprechen somit denen der Funktion best_move_minimax()

In [ ]:
def minimax(state, player, depth = 5):
    return(best_move_minimax(state, player, depth))

## Alpha-Beta-Pruning
Das Alpha-Beta-Pruning ist, wie im Rahmen des Minimax-Algorithmus schon erwähnt eine Verbesserung von Minimax.

In [ ]:
Cache = {}

`value_ab(State, player, alpha=-1, beta=1)` 

In [ ]:
def value_ab(state, player, alpha=-1, beta=1, depth = 6):
    global Cache
    state = to_tuple(state)
    if state in Cache:
        value, a, b = Cache[state]
        if a <= alpha and beta <= b:
            return value
        else:
            alpha = min(alpha, a)
            beta  = max(beta , b)
            value   = alphaBeta(state, player, alpha, beta, depth=depth)
            Cache[state] = value, alpha, beta
            return value
    else:
        value = alphaBeta(state, player, alpha, beta, depth=depth)
        Cache[state] = value, alpha, beta
        return value

`alphaBeta(State, player, alpha, beta)`

In [ ]:
def alphaBeta(state, player, alpha, beta, depth):
    state = to_list(state)
    if finished(state):
        return utility(state, player)
    if depth == 0:
        return heuristic(state, player)
    value = alpha
    for ns in next_states(state, player):
        value = max(value, -value_ab(ns, opponent(player), -beta, -alpha, depth = depth-1))
        if value >= beta:
            return value
        alpha = max(value, alpha)
    return value

In [ ]:
def best_move_ab(state, player, depth = 6):
    ns         = next_states(state, player)
    moves = [(-value_ab(s, opponent(player), depth), s) for s in ns]
    (best_value, best_state) = max(moves, key=lambda x:x[0])
    return best_value, best_state

Die Funktion `alpha_beta_pruning(State, Player)` wurde erstellt, um die Funktion `best_move(State, player)` nach Außen eindeutiger von `minimax(State, Player)` abzugrenzen.


In [ ]:
def alpha_beta_pruning(state, player, depth = 6):
    return(best_move_ab(state, player, depth = depth))

## Funktionstests:

In [ ]:
#import time
#start = time.time()
#state = [[4, 5], [[0, 2, 0, 0, 2, 0, 0, 0], [2, 1, 1, 0, 0, 0, 0, 0], [2, 1, 1, 1, 0, 0, 0, 0]]]
#print(alpha_beta_pruning(state, 2, 5))
#end = time.time()
#print(str(end-start)+'sec')

In [ ]:
#import time
#start = time.time()
#state = [[4, 5], [[0, 2, 0, 0, 2, 0, 0, 0], [2, 1, 1, 0, 0, 0, 0, 0], [2, 1, 1, 1, 0, 0, 0, 0]]]
#print(minimax(state, 2, 5))
#end = time.time()
#print(str(end-start)+'sec')

In [ ]:
#import time
#start = time.time()
#state = [[5, 6], [[0, 0, 0, 2, 0, 0, 0, 0], [0, 1, 1, 1, 2, 0, 2, 0], [0, 1, 0, 0, 0, 0, 0, 0]]]
#print(alpha_beta_pruning(state, 2, 5))
#end = time.time()
#print(str(end-start)+'sec')